In [1]:
import numpy as np
import pandas as pd
import re
import string

In [2]:
txt = 'great product!'

In [3]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')

    return text

In [4]:
with open ('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [5]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
def preprocess_txt(text):
    data = pd.DataFrame([text], columns=['text'])

    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuation)
    data['tweet'] = data['tweet'].str.replace('\s+', ' ', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Asus\AppData\Local\Temp\ipykernel_19904\2215632178.py:7: SyntaxWarning: invalid escape sequence '\s'
  data['tweet'] = data['tweet'].str.replace('\s+', ' ', regex=True)
